In [2]:
import pickle
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset

In [3]:
data_df = pd.read_csv("../data/train/train.tsv", sep="\t", header=None)
data_df

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
8995,wikipedia-5414-12-15-21-0-4,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,15,21,2002년,0,4,관계_없음
8996,wikipedia-10384-4-12-14-0-1,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,12,14,일본,0,1,단체:본사_국가
8997,wikipedia-25913-6-8-10-93-106,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,8,10,선무원종공신(宣武原從功臣),93,106,인물:직업/직함
8998,wikitree-12062-15-0-3-46-47,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,0,3,북미,46,47,관계_없음


In [21]:
label_keys = [k for k in data_df[8].value_counts().keys()]
label_counts = [v for v in data_df[8].value_counts().values]
print(len(data_df[8].value_counts()))
print(label_keys)
print(label_counts)

42
['관계_없음', '단체:구성원', '인물:소속단체', '인물:직업/직함', '단체:상위_단체', '단체:별칭', '인물:출신성분/국적', '인물:동료', '단체:제작', '인물:부모님', '단체:본사_도시', '단체:본사_국가', '인물:별칭', '인물:배우자', '인물:자녀', '단체:하위_단체', '단체:창립일', '인물:기타_친족', '인물:제작', '인물:형제/자매/남매', '인물:사망_일시', '단체:창립자', '단체:모회사', '인물:출생_일시', '인물:거주_국가', '인물:거주_도시', '단체:본사_주(도)', '단체:정치/종교성향', '인물:종교', '단체:해산일', '인물:거주_주(도)', '인물:용의자', '단체:주주', '인물:학교', '단체:구성원_수', '인물:출생_국가', '인물:사망_원인', '인물:나이', '단체:자회사', '인물:출생_도시', '인물:사망_도시', '인물:사망_국가']
[4440, 815, 680, 553, 335, 231, 209, 186, 165, 164, 136, 115, 103, 92, 84, 80, 67, 58, 56, 52, 45, 45, 44, 36, 27, 26, 23, 18, 15, 15, 12, 11, 11, 9, 9, 8, 7, 5, 5, 4, 3, 1]


# 데이터 랜덤 샘플링

뽑고자 하는 데이터보다 데이터 수가 많은 경우 그냥 sample로 뽑아쓰면 됨.

In [16]:
label_0_data_df = data_df.loc[data_df[8] == label_keys[0], :]
label_0_data_df

,0,1,2,3,4,5,6,7,8
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
5,wikitree-7590-2-33-35-19-21,이와 관련해 AFCD는 대학 소속 전문가와 세계동물보건기구(OIE)가 사람과 동물 ...,OIE,33,35,전문가,19,21,관계_없음
9,wikitree-48285-0-17-21-86-88,"김동양 NH투자증권 연구원은 “삼성SDS의 3분기 매출액은 2조6604억원, 영업이...",삼성SDS,17,21,10%,86,88,관계_없음
11,wikipedia-20141-6-11-15-38-42,"이후 구 공화당계와 유신정우회계 인사들은 김종철, 이만섭을 중심으로 한국국민당을 조...",유신정우회,11,15,한국국민당,38,42,관계_없음
14,wikipedia-26351-26-43-48-22-28,"쇠뇌를 소지한 병사가 실전에 임한 것은 제3차 십자군 원정 때인데, 사자심왕 리처드...",리처드 1세,43,48,제3차 십자군,22,28,관계_없음
...,...,...,...,...,...,...,...,...,...
8990,wikipedia-16106-146-17-20-0-3,사나다군과 모리군이 어떻게해서 이에야스 본인에까지 육박한 것에 대해 여러 설이 있어...,이에야스,17,20,사나다군,0,3,관계_없음
8991,wikipedia-23059-520-24-26-47-49,김영삼 전 대통령은 2012년 7월 11일 김문수 새누리당 대선 경선후보의 예방에 ...,김문수,24,26,박근혜,47,49,관계_없음
8992,wikipedia-6508-117-0-4-9-10,롱스트리트는 블랙장군의 휘하로 원군을 보낸다면 로즈크란즈의 부대를 오하이오주 방면으...,롱스트리트,0,4,장군,9,10,관계_없음
8995,wikipedia-5414-12-15-21-0-4,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,15,21,2002년,0,4,관계_없음


In [17]:
label_0_data_df.sample(n=300)

,0,1,2,3,4,5,6,7,8
1763,wikitree-42841-5-0-1-34-36,"임택 동구청장은 “이번 강의가 어르신들의 자존감을 회복하고, 지역민과 소통하며 인생...",임택,0,1,지역민,34,36,관계_없음
7256,wikitree-10686-13-4-8-13-15,앞으로 혁신추진위에서는 제9차 시정혁신 권고문을 전달받은 시로부터 3개월 이내에 세...,혁신추진위,4,8,제9차,13,15,관계_없음
6445,wikipedia-29593-1-18-26-43-50,원래는 영화 감독을 지망하였으나 다쓰노코 프로덕션의 《독수리 오형제》를 보고 애니메...,다쓰노코 프로덕션,18,26,애니메이션 제작,43,50,관계_없음
4851,wikipedia-21871-8-14-15-6-7,"2001년 북한과 수교한 독일은 북핵 문제가 해결되지 않는 한, 인도적 지원 외에 ...",독일,14,15,북한,6,7,관계_없음
6007,wikitree-18097-9-0-2-13-15,경찰청과 소방안전본부도 확진자가 발생하면 위치추적 등을 협조하고 응급환자 및 검체 ...,경찰청,0,2,확진자,13,15,관계_없음
...,...,...,...,...,...,...,...,...,...
3525,wikitree-2629-1-0-1-13-17,김산 무안군수는 22일 코로나19 전남6번 확진자 발생에 따른 입장문을 통해 “군민...,김산,0,1,코로나19,13,17,관계_없음
3776,wikitree-53763-0-34-34-4-6,16일 전북도에 따르면 전북도청 전 규제개혁팀장으로 근무했던 박 사무관이 지역현안 ...,박,34,34,전북도,4,6,관계_없음
2681,wikipedia-1593-64-21-23-7-9,검찰은 특히 정형근 한나라당 의원이 '국정원의 도청 내역'이라며 공개한 문건은 글 ...,국정원,21,23,정형근,7,9,관계_없음
7870,wikitree-27106-4-0-2-40-49,이용섭 광주광역시장이 20일 오후 광산구 공군 제1전투비행단에서 열린 '2019년도...,이용섭,0,2,2019년도 4분기,40,49,관계_없음


뽑고자 하는 데이터보다 데이터 수가 적은 경우 중복으로 뽑을 수 있게 설정해주면 됨.

In [18]:
label_41_data_df = data_df.loc[data_df[8] == label_keys[41], :]
label_41_data_df

,0,1,2,3,4,5,6,7,8
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가


In [20]:
label_41_data_df.sample(n=300, replace=True)

,0,1,2,3,4,5,6,7,8
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
...,...,...,...,...,...,...,...,...,...
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
